In [ ]:
# TODOS
# Add Rician objective function for LSQ fitting.
# rewrite for experiment

"""
(c) Stefano B. Blumberg and Paddy J. Slator, do not redistribute or modify

Code to replicate the ADC experiment (alongside matlab code - maybe translate to python?) <Add paper link>

Overview for cells:
    - Choose data size splits 2
    - Generate data examples 3-A/B/C
    - Data format for JOFSTO 4
    - Option to pass data directly, or save to disk and load 5-A/B
    - JOFSTO hyperparameters 6,7,8
"""

In [ ]:
########## (1)
# Import modules, see requirements.txt for jofsto requirements, set global seed

import numpy as np
import os, yaml
from jofsto_code.jofsto_main import return_argparser, run

import matplotlib.pyplot as plt

np.random.seed(0)  # Random seed for entire script

In [ ]:
#Directories and filenames to save data (Replace with location of JOFSTO code - possible to get this automatically?)
basedir = '/Users/paddyslator/python/ED_MRI/'

In [ ]:
########## (2)
# Data split sizes

n_train = 1000  # No. training voxels, reduce for faster training speed
n_val = n_train // 10  # No. validations set voxels
n_test = n_train // 10  # No. test set voxels

In [ ]:
########## (3-A)
# Create dummy, randomly generated (positive) data

# C_bar = 220
# M = 12  # Number of input measurements \bar{C}, Target regressors
# rand = np.random.lognormal  # Random genenerates positive
# train_inp, train_tar = rand(size=(n_train, C_bar)), rand(size=(n_train, M))
# val_inp, val_tar = rand(size=(n_val, C_bar)), rand(size=(n_val, M))
# test_inp, test_tar = rand(size=(n_test, C_bar)), rand(size=(n_test, M))


# #########
# #Generate data using an ADC model
maxb = 5
minb = 0
nb = 192

C_bar = nb

bvals = np.linspace(minb,maxb,nb)

def adc(D,bvals):
    signals = np.exp(-bvals*D)
    return signals

n_samples = n_train + n_val + n_test
minD = 0.1
maxD = 3
parameters = np.random.uniform(low=minD,high=maxD,size=(n_samples,1))

signals = np.zeros((n_samples,nb),dtype = np.float32)
for i in range(0,n_samples):
    signals[i,:] = adc(parameters[i],bvals)



In [ ]:
#add noise
def add_noise(data, scale=0.1):
    data_real = data + np.random.normal(scale=scale, size=np.shape(data))
    data_imag = np.random.normal(scale=scale, size=np.shape(data))
    data_noisy = np.sqrt(data_real**2 + data_imag**2)

    return data_noisy

signals = add_noise(signals)



In [ ]:
########## (4)
# Load data into JOFSTO format

# Data in JOFSTO format, \bar{C} measurements, M target regresors
data = dict(
    train=signals[0:n_train,:],  # Shape n_train x \bar{C}
    train_tar=parameters[0:n_train,:],  # Shape n_train x M
    val=signals[n_train:(n_train + n_val),:],  # Shape n_val x \bar{C}
    val_tar=parameters[n_train:(n_train + n_val),:],  # Shape n_val x M
    test=signals[(n_train + n_val):(n_train + n_val + n_test),:],  # Shape n_test x \bar{C}
    test_tar=parameters[(n_train + n_val):(n_train + n_val + n_test),:],  # Shape n_test x M
)

#with open(os.path.dirname(__file__) + "/base.yaml", "r") as f:
#with open("/home/blumberg/Bureau/z_Automated_Measurement/Code/base.yaml", "r") as f:
with open(os.path.join(basedir, "base.yaml"), "r") as f:
    args =  yaml.safe_load(f)

In [ ]:
                        ########## (5-A)
# Option to save data to disk, and JOFSTO load
data_fil = os.path.join(basedir,'adc_simulations.npy')
#data_fil = "/home/blumberg/Bureau/z_Automated_Measurement/Output/paddy/adc_simulations.npy"
#data_fil = "/Users/paddyslator/python/ED_MRI/adc_simulations.npy"  # Add path to save file
np.save(data_fil, data)
print("Saving data as", data_fil)
pass_data = None

args["--data_fil"] = data_fil


########## (5-B)
# Option to pass data to JOFSTO directly

pass_data = data

In [ ]:
########## (6)
# Simplest version of JOFSTO, modifying the most important hyperparameters


# Decreasing feature subsets sizes for JOFSTO to consider
args["C_i_values"] = [C_bar, C_bar // 2, C_bar // 4, C_bar // 8, C_bar // 16]

# Feature subset sizess for JOFSTO evaluated on test data
args["C_i_eval"] = [C_bar // 2, C_bar // 4, C_bar // 8, C_bar // 16]

# Scoring net C_bar -> num_units_score[0] -> num_units_score[1] ... -> C_bar units
args["num_units_score"] = [1000, 1000]

# Task net C_bar -> num_units_task[0] -> num_units_task[1] ... -> M units
args["num_units_task"] = [1000, 1000]

args["out_base"] =  os.path.join(basedir, "test1")  #"/Users/paddyslator/python/ED_MRI/test1" #"/home/blumberg/Bureau/z_Automated_Measurement/Output/paddy"
args["proj_name"] = "adc"
args["run_name"] = "test"

args["save_output"] = True

#args["total_epochs"] = 1000


JOFSTO_output = run(args=args, pass_data=pass_data)


In [ ]:
#import jofsto_code.utils.load_results as load_results
#JOFSTO_output = np.load(os.path.join(basedir,args["out_base"],args["proj_name"],"results", args["run_name"] + "_all.npy"), allow_pickle=True).item()

In [ ]:
#load the CRLB optimised protocol
#import scipy.io as sio
#CRLB_ADC = sio.loadmat('/Users/paddyslator/MATLAB/adc_crlb/crlb_adc_optimised_protocol.mat')
bvals_CRLB_ADC = np.loadtxt(os.path.join(basedir, "crlb_code/crlb_adc_optimised_protocol.txt"))
#bvals_CRLB_ADC = np.squeeze(CRLB_ADC['b_opt'])


In [ ]:
#plot the JOFSTO and CRLB b-values
#all super-design b-values
Dtest = 1
plt.plot(bvals,adc(Dtest,bvals),'o')
#JOFSTO chosen b-values
C_last = JOFSTO_output["C_i_eval"][-1]
print(JOFSTO_output[C_last]['measurements'])
plt.plot(bvals[JOFSTO_output[C_last]['measurements']],adc(Dtest,bvals[JOFSTO_output[C_last]['measurements']]),'x')

#CRLB chosen b-values
plt.plot(bvals_CRLB_ADC,adc(Dtest,bvals_CRLB_ADC),'o')


In [ ]:
#JOFSTO_output[12]["test_output"].shape
#test_tar.shape
test_tar = data["test_tar"][:,0]
np.corrcoef(test_tar,JOFSTO_output[12]["test_output"][:,0])

In [ ]:
from scipy.optimize import minimize

#fit the ADC model on the full acquisition, JOFSTO acquisition, CRLB acquisition

#for the super design use the test data signals 
signals_super = signals[(n_train + n_val):(n_train + n_val + n_test),:]
#for jofsto use the reconstructed data
signals_jofsto = signals[(n_train + n_val):(n_train + n_val + n_test),JOFSTO_output[C_last]['measurements']] 



#simulate data for the CRLB acquisition - as don't have test data at these b-values
signals_crlb = np.zeros((n_samples,len(bvals_CRLB_ADC)))

#use the ground truth parameters from the test dataset
parameters = test_tar

#signals_super = np.zeros((n_samples,len(bvals)))
#signals_jofsto = np.zeros((n_samples,len(bvals[JOFSTO_output[C_last]['measurements']])))

#simulate some new parameters
#parameters = np.random.uniform(low=minD,high=maxD,size=n_samples)


for i in range(0,n_test):
    signals_crlb[i,:] = add_noise(adc(parameters[i],bvals_CRLB_ADC),scale=0.1)
#    signals_super[i,:] = add_noise(adc(parameters[i],bvals),scale=0.1)
#    signals_jofsto[i,:] = add_noise(adc(parameters[i],bvals[JOFSTO_output[C_last]['measurements']]),scale=0.1)
    

def objective_function(D,bvals,signals):
    return np.mean((signals - adc(D,bvals))**2)
    
    
# def adc(D,bvals):
#     signals = np.exp(-bvals*D)
#     return signals

Dstart = 1

fitted_parameters_crlb = np.zeros(n_test)
fitted_parameters_super = np.zeros(n_test)
fitted_parameters_jofsto = np.zeros(n_test)

for i in range(0,n_test):
    fitted_parameters_crlb[i] = minimize(objective_function, Dstart, args=(bvals_CRLB_ADC,signals_crlb[i,:]),method='Nelder-Mead').x
    fitted_parameters_super[i] = minimize(objective_function, Dstart, args=(bvals,signals_super[i,:]),method='Nelder-Mead').x
    fitted_parameters_jofsto[i] = minimize(objective_function, Dstart, args=(bvals[JOFSTO_output[C_last]['measurements']],signals_jofsto[i,:]),method='Nelder-Mead').x

In [ ]:
def mean_squared_error(x,y):
    return ((x - y)**2).mean(axis=0)
    

plt.plot(parameters,fitted_parameters_crlb,'o',markersize=5)
plt.plot(parameters,fitted_parameters_super,'v',markersize=5)
plt.plot(parameters,fitted_parameters_jofsto,'x',markersize=10)
plt.plot(test_tar,JOFSTO_output[12]["test_output"][:,0],'^')

plt.plot((minD,maxD),(minD,maxD),'k',markersize=5)

plt.xlabel('ground truth D ($\mu$m$^2$s$^{-1}$)')
plt.xlabel('predicted D ($\mu$m$^2$s$^{-1}$)')

print("CRLB correlation: " + str(np.corrcoef(parameters,fitted_parameters_crlb)[0,1]))
print("super correlation " + str(np.corrcoef(parameters,fitted_parameters_super)[0,1]))
print("JOFSTO LSQ correlation " + str(np.corrcoef(parameters,fitted_parameters_jofsto)[0,1]))
print("JOFSTO NN correlation " + str(np.corrcoef(test_tar,JOFSTO_output[12]["test_output"][:,0])[0,1]))


print("CRLB MSE: " + str(mean_squared_error(parameters,fitted_parameters_crlb)))
print("super MSE " + str(mean_squared_error(parameters,fitted_parameters_super)))
print("JOFSTO LSQ MSE " + str(mean_squared_error(parameters,fitted_parameters_jofsto)))
print("JOFSTO NN MSE " + str(mean_squared_error(test_tar,JOFSTO_output[12]["test_output"][:,0])))




In [ ]:
########## (7)
# Modify more JOFSTO hyperparameters, less important, may change results

In [ ]:

########## (8)
# Deep learning training hyperparameters for inner loop